# PyTorch Hooks

### Initialization

In [2]:
import numpy as np

import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

from collections import OrderedDict 

from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

In [3]:
seed_value = 0
np.random.seed(seed_value)
torch.manual_seed(seed_value)

### Model

In [4]:
class NewModel(nn.Module):
    def __init__(self, output_layers, *args):
        super().__init__(*args)
        self.output_layers = output_layers
        
        # Pretrained model we will be using
        self.pretrained = resnet50(weights=ResNet50_Weights.DEFAULT)
        
        # Where the output of the hooks will be stored
        self.selected_out = OrderedDict()

        # Register the forward hook on the given output layers
        # Forward Hook is triggered every time after the method foward of the Pytorch AutoGrad Function grad_fn
        # We can modify the output by returning the modified output from the hook. 
        # Using forward_pre_hook the user can modify the input but returning the modified input value as a tuple or just a single modified value in the hook.
        self.fhooks = []
        for i, l in enumerate(list(self.pretrained._modules.keys())):
            if i in self.output_layers:
                self.fhooks.append(getattr(self.pretrained,l).register_forward_hook(self.custom_forward_hook(l)))
        
        self.bhooks = []

    def custom_forward_hook(self, layer_name):
        def hook(module, input, output):
            self.selected_out[layer_name] = output

        return hook

    def forward(self, x):
        out = self.pretrained(x)
        return out, self.selected_out

### Testing

In [5]:
# 7 -> layer4
# 8 -> avgpool
model = NewModel(output_layers = [7,8])

# Random array with the dimension of the layer 4
target_ft = torch.rand((2048,8,8))

learning_rate = 0.00001
params = [p for p in model.parameters() if p.requires_grad]
optimizer = Adam(params, lr=learning_rate)
lr_scheduler = StepLR(optimizer, step_size=1, gamma=0.9)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\diogo/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100.0%


tensor([[[0.6515, 0.3627, 0.9136,  ..., 0.9496, 0.1896, 0.8063],
         [0.1327, 0.1769, 0.8374,  ..., 0.2746, 0.0749, 0.2727],
         [0.9443, 0.6604, 0.7327,  ..., 0.4276, 0.6980, 0.0976],
         ...,
         [0.3596, 0.6810, 0.3878,  ..., 0.1851, 0.2005, 0.7427],
         [0.5954, 0.5774, 0.9878,  ..., 0.1426, 0.7741, 0.1874],
         [0.6988, 0.8268, 0.1948,  ..., 0.0867, 0.1275, 0.6169]],

        [[0.5213, 0.2335, 0.1328,  ..., 0.8023, 0.3130, 0.7500],
         [0.9003, 0.7355, 0.3945,  ..., 0.9670, 0.0738, 0.5304],
         [0.3885, 0.4687, 0.2886,  ..., 0.7229, 0.3495, 0.2906],
         ...,
         [0.2004, 0.1805, 0.9476,  ..., 0.9128, 0.7163, 0.7619],
         [0.4754, 0.2336, 0.4802,  ..., 0.9757, 0.3688, 0.0140],
         [0.6812, 0.4794, 0.3241,  ..., 0.8627, 0.9429, 0.8025]],

        [[0.4981, 0.2675, 0.1235,  ..., 0.6680, 0.3709, 0.1518],
         [0.9828, 0.0543, 0.7974,  ..., 0.3167, 0.8277, 0.9925],
         [0.1653, 0.5710, 0.0242,  ..., 0.0428, 0.1086, 0.

In [6]:
# Define the Image
batch_size = 1
channels = 3
height = 256
width = 256

image_array = np.random.randint(0, 256, size=(batch_size, channels, height, width)).astype('float32')
x = torch.from_numpy(image_array)

out, layerout = model(x)
layer4out = layerout['layer4']